In [ ]:
######pyspark实现kmeans

In [1]:
from pyspark import SparkContext
from pyspark.sql import SparkSession

#####使用本地的pyspark
sc = SparkContext( 'local', 'pyspark')

In [2]:
#####指定spark-warehouse
spark = SparkSession.builder\
    .master('local')\
    .appName('spark_mllib')\
    .config('spark.sql.warehouse.dir', 'file:///F:/workspace/work/project/spark/spark-warehouse')\
    .getOrCreate()

In [3]:
from numpy import array
from math import *

# Load and parse the data file.
path = r'C:\spark-2.0.0-bin-hadoop2.6'

data = sc.textFile(path+"\\"+"data/mllib/kmeans_data.txt")

parsedData = data.map(lambda line: array([float(x) for x in line.split(' ')]))

Explore the data

In [6]:
type(parsedData)
parsedData.collect()

[array([ 0.,  0.,  0.]),
 array([ 0.1,  0.1,  0.1]),
 array([ 0.2,  0.2,  0.2]),
 array([ 9.,  9.,  9.]),
 array([ 9.1,  9.1,  9.1]),
 array([ 9.2,  9.2,  9.2])]

In [7]:
# Build the model (cluster the data)
from pyspark.mllib.clustering import KMeans, KMeansModel

clusters = KMeans.train(parsedData, 2, maxIterations=10,
                        runs=10, initializationMode="random")

D:\Anaconda2\lib\site-packages\pyspark\mllib\clustering.py:347: UserWarning: The param `runs` has no effect since Spark 2.0.0.
  warnings.warn("The param `runs` has no effect since Spark 2.0.0.")


In [8]:
# Evaluate clustering by computing Within Set Sum of Squared Errors
def error(point):
    center = clusters.centers[clusters.predict(point)]
    return sqrt(sum([x**2 for x in (point - center)]))

WSSSE = parsedData.map(lambda point: error(point)).reduce(lambda x, y: x + y)
print "Within Set Sum of Squared Error = " + str(WSSSE)

Within Set Sum of Squared Error = 0.692820323028


In [9]:
# Save and load model
#save the model
clusters.save(sc, "PythonKMeansExample")

In [ ]:
#load the model
sameModel = KMeansModel.load(sc, "PythonKMeansExample")